# トレンドレコメンド（仮）

## 概要

- 世間で話題になっているトレンドを抽出し、トレンドに関連した商品を自動抽出する
- 使用したAPI （TwitterAPI, NewsAPI, 楽天API）

## 処理の流れ

1. 以下のいずれかの方法により、トレンドワードを取得する
- GoogleトレンドのWebページをクローリング・スクレイピングし、Google検索で急上昇中のワードを取得する（https://trends.google.co.jp/trends/trendingsearches/realtime?geo=JP&category=all）
- TwitterAPIを使用し、Twitterでトレンドになっているワードを取得する
- NewsAPIを使用し、国内ニュースでトレンドになっているニュースを取得する（これはワードではなく、ニュース記事を取得）
<br>
※ 各処理で必要となる処理は関数化済みだが、今回はGoogleトレンドのパターンで実験する


2. トレンドワードに関連するニュース記事をNewsAPIを使用して取得する
- 1で取得したトレンドワードでニュース記事を検索する
- ニュース記事をタイトルと概要を取得する（本文取得はAPIではできない）

3. トレンドワードに関連するツイートをTwitterAPIを使用して取得する
- 取得したトレンドワードでツイートを検索する

4. トレンドワード・ニュース記事（複数）・ツイート（10件）を一つのテキストにまとめる
5. 楽天APIを使用し、楽天ブックスで販売する書籍のタイトル・著者・概要を取得する
- 一度に楽天APIで取得できる商品数には制限がある為、事前にAPIを一日叩き続けて取得できた10万件近い書籍のデータを使用する
- 書籍データはpandasのデータとして保持し、pickleファイル化しておく（実際はDBに保存しておきたい）

6. 5で取得した書籍情報（タイトル・著者・概要）を一つのテキストにまとめる

7. 4と6をMecabで形態素解析し、名詞だけを取得する

8. TFIDFのベクトル化して、トレンドに関連するテキスト（トレンドワード・ニュース記事・ツイート）に一番類似する商品をCOS類似度を算出し求める

## 以下、処理内容

In [1]:
# 必要なライブラリをインストール（AWSで環境をゼロから作り直す時必要）
# !pip install newsapi-python
# !pip install twitter
# !pip install selenium
# !pip install feedparser

### 前処理

In [1]:
# 各APIで必要なライブラリをインポート
from newsapi import NewsApiClient
from twitter import *

In [25]:
# その他、必要なライブラリをインポート
import datetime
import re
from selenium import webdriver
import time
import feedparser
import random
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
# API_KEYの設定ファイルを読み込み
# ※ APIを使用する為に必要となるキーとパスをまとめた設定ファイルを別途作成し、それを呼び出している
import configparser
config = configparser.ConfigParser()
config.read('config.ini')

['config.ini']

In [21]:
# postgresqlに接続する為のライブラリ
import psycopg2
from sqlalchemy import create_engine

### news_api

In [4]:
# news_apiから、検索したキーワードに関連するニュース情報を返す関数
def keyword_news_search(\
                         api_key = config['NEWS_API']['KEY'],\
                         search_word = '', \
                         start_date = datetime.datetime.now().strftime('%Y-%m-%d'),\
                         end_date = datetime.datetime.now().strftime('%Y-%m-%d'), \
                         sort_type = 'popularity'\
                        ):
    
    # news_apiを初期化
    news_api_search = NewsApiClient(api_key=config['NEWS_API']['KEY'])
    
    # search_wordで指定したキーワードに関するニュースを検索する
    result_search_news = news_api_search.get_everything(qintitle=search_word, from_param=start_date ,to=end_date, sort_by=sort_type)
    
    return result_search_news

In [5]:
# news_apiから、トップニュース情報を返す関数
def headlines_news(\
                         api_key = config['NEWS_API']['KEY'],\
                         target_category = 'entertainment', \
                         target_country = 'jp'
                        ):
    
    # news_apiを初期化
    news_api_headlines = NewsApiClient(api_key=config['NEWS_API']['KEY'])
    
    # 指定したオプションに従ったトップニュースを検索する     
    # Possible options: business entertainment general health science sports technology . 
    # Note: you can't mix this param with the sources param.
    result_headlines_news = news_api_headlines.get_top_headlines(category=target_category, country=target_country)
    
    return result_headlines_news

### twitter_api

In [6]:
# twitter_apiから、ツイートトレンドの単語を返す関数
def twitter_trends_search(\
                          CK = config['TWITTER_API']['CONSUMER_KEY'],\
                          CS = config['TWITTER_API']['CONSUMER_SECRET'],\
                          AT = config['TWITTER_API']['ACCESS_TOKEN'],\
                          AS = config['TWITTER_API']['ACCESS_TOKEN_SECRET'],\
                          area_id = 23424856\
                         ):
    
     # twitter_apiを初期化
    twitter_api_trends = Twitter(auth = OAuth(AT,AS,CK,CS))
        
     # 指定したエリアのツイッタートレンドを検索する
    twitter_trends_results = twitter_api_trends.trends.place(_id = area_id)
                    
    return twitter_trends_results

In [7]:
# twitter_apiから、検索したキーワードに関連するツイートを返す関数
def twitter_search(\
                          CK = config['TWITTER_API']['CONSUMER_KEY'],\
                          CS = config['TWITTER_API']['CONSUMER_SECRET'],\
                          AT = config['TWITTER_API']['ACCESS_TOKEN'],\
                          AS = config['TWITTER_API']['ACCESS_TOKEN_SECRET'],\
                          search_keyword = 'twitter',\
                          search_lang = 'ja',\
                          search_type = 'mixed',\
                          search_count = 100\
                         ):
    
     # twitter_apiを初期化
    twitter_api_search = Twitter(auth = OAuth(AT,AS,CK,CS))
        
     # 指定したエリアのツイッタートレンドを検索する
    search_results = twitter_api_search.search.tweets(q=search_keyword, lang=search_lang, result_type=search_type, count=search_count)
                    
    return search_results

### Googleトレンド（クローリング・スクレイピング）

In [8]:
def google_trends_search(\
                        base_url = 'https://trends.google.co.jp/trends/trendingsearches/realtime',\
                        search_geo = 'JP',\
                        search_category = 'all',
                        ):
    
    try:

        # 検索するURLを作成する
        url = '{i}?geo={j}&category={k}'.format(i=base_url, j=search_geo, k=search_category)

        # ブラウザを開く
        driver = webdriver.Chrome()

        # Googleトレンドの検索TOP画面を開く （表示サイズも指定しておく）
        driver.get(url)
        driver.set_window_size(2500,1000)

        # 2〜4秒間、ページが開かれるのを待つ
        time.sleep(random.randint(2,4))

        # TOP画面より、必要な要素のデータを取得する
        trends_keywords = driver.find_elements_by_class_name('title')
        summary_text = driver.find_elements_by_class_name('summary-text')
        source_text = driver.find_elements_by_class_name('source-and-time')

        # 全ての要素が取得できた場合、処理を進める
        while bool(len(trends_keywords) == 0 or  len(summary_text) == 0 or len(source_text) == 0):
            time.sleep(10)
            trends_keywords = driver.find_elements_by_class_name('title')
            summary_text = driver.find_elements_by_class_name('summary-text')
            source_text = driver.find_elements_by_class_name('source-and-time')
        else:
            trends_keyword = []
            trends_abst = []
            inyo_url = []
            inyo_site = []
            for i in range(len(trends_keywords)):
                trends_keyword.append(trends_keywords[i].text)
                trends_abst.append(summary_text[i].text)
                inyo_url.append(summary_text[i].find_element_by_tag_name("a").get_attribute("href"))
                inyo_site.append(source_text[i].text.split()[0])

        # 各トレンドごとに詳細情報を取得する為、要素をクリックし必要なHTMLを表示させる（その為のクリックする要素を取得している）
        trend_click = driver.find_elements_by_class_name('feed-item')

        # 蓄積用のリストを用意しておく
        google_trend_news_list = []
        google_trend_words_list = []

        # トレンドごとに詳細情報を取得する為、要素をクリックし必要なHTMLを表示させる
        for trend in trend_click:
            trend.click()

            # 1秒間、ページが開かれるのを待つ
            time.sleep(3)
            
            # 引用ニュースを取得
            google_trend_news_tmp = driver.find_elements_by_class_name('item-title')
            google_trend_news = ''
            for news in google_trend_news_tmp:
                google_trend_news = google_trend_news + news.text

            # 関連ワードを取得
            google_trend_words_tmp = driver.find_elements_by_class_name('list')
            google_trend_words = google_trend_words_tmp[0].text.replace(" ","")

            # リストに追加
            google_trend_news_list.append(google_trend_news)
            google_trend_words_list.append(google_trend_words) 

        driver.quit()
    
    except:
        pass
#         driver.quit()
    
    google_trend_data = {'goole_keyword': trends_keyword, 'google_abst': trends_abst, 'google_url': inyo_url, 'google_site': inyo_site, \
                        'google_news':google_trend_news_list, 'google_words':google_trend_words_list}
    
    return google_trend_data

In [9]:
google_trends = google_trends_search()

## 必要な処理は用意できたので、ここからレコメンドを作っていく。

#### ちょっとしたデータ加工

In [10]:
google_trend_word = []
for i in google_trends['goole_keyword']:
    google_trend_word.append(i.split(' • '))

In [11]:
google_trend_news = google_trends['google_news']
google_trend_words = google_trends['google_words']

#### 【NesAPIを使用】トレンドワードに基づいて、関連ニュースを取得する

In [12]:
sample_news = []
for j in google_trend_word:
    sample_news.append(keyword_news_search(search_word = j[0]))
    time.sleep(1)

In [13]:
news_text = []
for k in sample_news:
    news_text_tmp = ''
    for o in k['articles']:
        news_text_tmp += (o['title'] +","+ str(o['description']))
    news_text.append(news_text_tmp)

#### 【twitterAPIを使用】トレンドワードに基づいて、関連ツイートを取得する

In [14]:
twtter_text = []
for p in google_trend_word:
    twtter_text.append(twitter_search(search_keyword=p[0], search_count = 10))
    time.sleep(1)

In [15]:
twitter_text_list = []
for q in twtter_text:
    
    twitter_text_tmp = ''
    for i in range(len(q["statuses"])):
        twitter_text_tmp += q["statuses"][i]["text"]
        
    twitter_text_list.append(twitter_text_tmp)

#### googleトレンドのページに記載されているテキストは重みをつける（✖️3）

In [ ]:
#　ちょっとデータ加工
google_trend_word_text = []
for r in google_trend_word:
    google_trend_word_text_tmp = ','.join(r)
    google_trend_word_text.append(google_trend_word_text_tmp)

In [20]:
test_text = []
for s in range(len(google_trend_word_text)):
    test_text.append((google_trend_word_text[s] + google_trend_news[s] + google_trend_words[s]) * 3 + news_text[s] + twitter_text_list[s])

## 楽天APIで取得済みの商品リストを持ってくる

#### 楽天APIで書籍情報を取得し続ける処理は別途作成し実行ずみ。.pickleのファイルにしているので、ここで読み込み使用する。

In [22]:
# PostgreSQL Server へ接続（必要情報は削除）
conn = psycopg2.connect('host= port= dbname= user= password=')

In [23]:
# データベースの接続情報（必要情報は削除）
connection_config = {
    'user': '',
    'password': '',
    'host': '',
    'port': '', 
    'database': ''
}
engine = create_engine('postgresql://{user}:{password}@{host}:{port}/{database}'.format(**connection_config))

In [188]:
# PostgreSQLに楽天APIから集計したデータを書き込む

# import pickle
# with open('df_all_duplicated.pickle','rb') as f:
#     book_list = pickle.load(f)
# book_list_db.to_sql('book', con=engine, if_exists='replace', index=False)

In [ ]:
# PostgreSQLにトレンド情報のデータを書き込む
df_trend = pd.DataFrame({'time': pd.to_datetime(datetime.datetime.now(), format='%y%m%d %H:%M'),\
                    'google_trend_words': google_trend_word_text,\
                  'google_trend_news': google_trend_news,\
                  'google_trend_words_related': google_trend_words,\
                  'news': news_text,\
                  'tweet': twitter_text_list})
df_trend.to_sql('trend', con=engine, if_exists='append', index=False)

In [123]:
# トレンド情報を見てみる
df_trend.head()

,time,google_trend_words,google_trend_news,google_trend_words_related,news,tweet
0,2021-01-16 01:57:20.546187,"新世紀エヴァンゲリオン,綾波 レイ,集結の園へ,残酷な天使のテーゼ",...Singing Cosplayer Hikari、綾波レイコスプレで歌うカバー動画「集...,エヴァンゲリオン\nエバンゲリオン\nエヴァンゲリオン映画\nエヴァ\nシンエヴァンゲリオン...,,1995年に📺TVシリーズ『新世紀エヴァンゲリオン』がスタートし、社会現象となりましたが、そ...
1,2021-01-16 01:57:20.546187,"新世紀エヴァンゲリオン,シン・エヴァンゲリオン劇場版:||,ヱヴァンゲリヲン新劇場版:破",【衝撃】ネットで「エヴァのガチ勢にしかわからないことリスト」が話題 / ...『シン・エヴァ...,エヴァンゲリオン\n金曜ロードショー\nエバンゲリオン\nエヴァンゲリオン映画\nエヴァ\n...,,1995年に📺TVシリーズ『新世紀エヴァンゲリオン』がスタートし、社会現象となりましたが、そ...
2,2021-01-16 01:57:20.546187,"LiSA,中居正広の金曜日のスマイルたちへ,鬼滅の刃,日本レコード大賞",LiSA、家出同然で上京、極貧生活･･･どん底の日々を支えた母の“教え”とは?...歌手・L...,lisa\nリサ\nlisa年齢\n鈴木達央\nlisa結婚\n紅蓮華\nソニーミュージック...,"金スマに超気になる女！ 人気歌手・LiSAが登場！,金スマに超気になる女！ 人気歌手・LiS...",今日もいい日だっ！\n#金スマ宮田さん本当に重大な役を、ありがとうございました。\n#金スマ...
3,2021-01-16 01:57:20.546187,"町田市,東京都,飛鳥病院",＜新型コロナ＞東京・町田の「飛鳥病院」で103人が集団感染東京・町田の病院で１０３人集団感染,厚切りジェイソン\n飛鳥病院\n戸田中央総合病院\n町田飛鳥病院\n町田市飛鳥病院\n相模原...,"小田急電鉄とJR東日本、オンデマンド交通サービス「E-バス」の実証運行を町田市内にて実施,小...",東京、新規感染２００１人 町田市の病院で１０３人 https://t.co/RF1kEY04...
4,2021-01-16 01:57:20.546187,"シン・エヴァンゲリオン劇場版:||,林原めぐみ,綾波 レイ",林原めぐみ、再延期シン・エヴァ「私も早く見たい」林原めぐみ、公開再延期のシン・エヴァ「ホント...,エヴァンゲリオン\n地震速報\nエバンゲリオン\nエヴァ\nシンエヴァンゲリオン\n緒方恵美...,「シン・エヴァンゲリオン劇場版」公開記念のSSD/HDDコラボモデルがバッファローから。数量...,この度の緊急事態宣言の発出を受け慎重に検討を重ねた結果、感染拡大の収束が最優先であると判断し...


In [24]:
# PostgreSQLに接続する
connection = psycopg2.connect(**connection_config)

In [28]:
# テーブルの全レコードを格納する
book_list = pd.read_sql(sql="SELECT * FROM book;", con=conn)

In [35]:
# データのカラム名
book_list.columns

Index(['limitedFlag', 'authorKana', 'author', 'subTitle', 'seriesNameKana',
       'title', 'subTitleKana', 'itemCaption', 'publisherName', 'listPrice',
       'isbn', 'largeImageUrl', 'mediumImageUrl', 'titleKana', 'availability',
       'postageFlag', 'salesDate', 'contents', 'smallImageUrl',
       'discountPrice', 'itemPrice', 'size', 'booksGenreId', 'affiliateUrl',
       'seriesName', 'reviewCount', 'reviewAverage', 'discountRate',
       'chirayomiUrl', 'itemUrl'],
      dtype='object')

In [39]:
# 本の冊数
len(book_list)

522155

In [38]:
# 先頭5行を確認
book_list.head()

,limitedFlag,authorKana,author,subTitle,seriesNameKana,title,subTitleKana,itemCaption,publisherName,listPrice,...,itemPrice,size,booksGenreId,affiliateUrl,seriesName,reviewCount,reviewAverage,discountRate,chirayomiUrl,itemUrl
0,0,"ミズシマ,シンジ",水島新司,,ショウネン チャンピオン コミックス,ドカベン スーパースターズ編（45）,,,秋田書店,0,...,460,コミック,001001001001,,少年チャンピオン・コミックス,18,3.94,0,,https://books.rakuten.co.jp/rb/11644701/
1,0,"ツカワキ,ナガヒサ/ナンジョウ,ヨシミ",塚脇永久/ナンジョウヨシミ,,ショウネン チャンピオン コミックス エクストラ,潮騒の凡（2）,,,秋田書店,0,...,693,コミック,001001001001,,少年チャンピオンコミックス エクストラ,0,0.0,0,,https://books.rakuten.co.jp/rb/15940896/
2,0,"ミズシマ,シンジ",水島新司,,ショウネン チャンピオン コミックス,ドカベン プロ野球編（46）,,,秋田書店,0,...,429,コミック,001001001001,,少年チャンピオンコミックス,0,0.0,0,,https://books.rakuten.co.jp/rb/1528580/
3,0,"ハマオカ,ケンジ",浜岡賢次,,シヨウネンチヤンピオンコミツクス,元祖！浦安鉄筋家族（6）,,,秋田書店,0,...,460,コミック,001001001001,,少年チャンピオンコミックス,2,4.0,0,,https://books.rakuten.co.jp/rb/1668655/
4,0,"ミズシマ,シンジ",水島新司,,ショウネン チャンピオン コミックス,ドカベン プロ野球編（52）,,,秋田書店,0,...,429,コミック,001001001001,,少年チャンピオンコミックス,5,4.4,0,,https://books.rakuten.co.jp/rb/1653839/


In [120]:
test_books = (book_list['title']+","+book_list['author']+",")*5+","+book_list['subTitle']+","+book_list['itemCaption']

In [30]:
df1 = pd.DataFrame()
df1['test'] = test_books

In [31]:
df2 = pd.DataFrame(test_text, columns=['test'])

In [42]:
df3 = df1

#### 自然言語処理で、テキストをベクトルに変換していく

In [44]:
from sklearn import preprocessing

In [45]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

In [46]:
import MeCab
import re

In [54]:
# MeCabの辞書にnelogdを指定する
m = MeCab.Tagger(r"-r C:\Users\kazum\Desktop\GS\trend_recommend\database_test\api\mecabrc-u")

In [55]:
# あまり関係のないと思われる数字を全て0に置き換える関数
def replace_number_to_zero(text):
    changed_text = re.sub(r'[0-9]+', "0", text) #半角
    changed_text = re.sub(r'[０-９]+', "0", changed_text) #全角
    return changed_text

# 数字を0に置換
df3['review_number_to_zero'] = df3['test'].map(replace_number_to_zero)

In [56]:
# 分かち書きした結果を返す関数
def leaving_space_between_words_column(text):
    
    nouns = []
        
    for line in m.parse(text).splitlines():
        if len(line.split()) > 0:
            if "名詞" in line.split()[-1]:
                nouns.append(line)
                
    meishi_list = []
    for str in nouns:
        try:
#             meishi_list.append(str.split()[2])
            meishi_list.append(str.split(",")[6])
        except:
            meishi_list.append("")
    
#     return ','.join(meishi_list)
    return ' '.join(meishi_list)

# 分かち書きしたカラムをdfに追加する
df3['lsbw'] = df3['review_number_to_zero'].map(leaving_space_between_words_column)

In [57]:
# 50万冊の書籍情報をベクトル化
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df3['lsbw'])

In [84]:
# トレンドのテキスト情報もベクトル化
test_item = vectorizer.transform(df2["test"])

In [126]:
# トレンドに関連した商品を抽出
for u in test_item:
    similarity = cosine_similarity(u, X)
    aaa = similarity[0].tolist()
    print('-----')
    for v in range(-1,-4,-1):
        max_value = sorted(aaa)[v]
        max_index = aaa.index(max_value)
        print(max_value)
        print(df3.iloc[max_index, :])
        print('-----')
    print('-----')
    print('-----')
    print('-----')
    base = base + 1

-----
0.25759045054754554
test                     【輸入楽譜】キングズ・シンガース - アンサンブル・シンギング・コレクション/無伴奏混声四部...
review_number_to_zero    【輸入楽譜】キングズ・シンガース - アンサンブル・シンギング・コレクション/無伴奏混声四部...
lsbw                     輸入 楽譜 * * アンサンブル Singing! コレクション * 無伴奏 混声 四 部 ...
Name: 450576, dtype: object
-----
0.24877509365837974
test                     新世紀エヴァンゲリオン エヴァ&エヴァ2アンソロジー,少年エース編集部,新世紀エヴァンゲリオ...
review_number_to_zero    新世紀エヴァンゲリオン エヴァ&エヴァ0アンソロジー,少年エース編集部,新世紀エヴァンゲリオ...
lsbw                     新世紀エヴァンゲリオン エヴァ * エヴァ * アンソロジー * 少年エース編集部 * 新世...
Name: 42906, dtype: object
-----
0.24133732215592923
test                     残酷な天使のテーゼ,,残酷な天使のテーゼ,,残酷な天使のテーゼ,,残酷な天使のテーゼ,,残酷...
review_number_to_zero    残酷な天使のテーゼ,,残酷な天使のテーゼ,,残酷な天使のテーゼ,,残酷な天使のテーゼ,,残酷...
lsbw                     残酷な天使のテーゼ * 残酷な天使のテーゼ * 残酷な天使のテーゼ * 残酷な天使のテーゼ ...
Name: 370938, dtype: object
-----
-----
-----
-----
-----
0.4291976371733539
test                     新世紀エヴァンゲリオン エヴァ&エヴァ2アンソロジー,少年エース編集部,新世紀エヴァンゲリオ...
review_number_to_zero    新世紀エヴァンゲリオン エヴァ

-----
0.5818497105985977
test                     綾瀬はるか meets Beautiful Athletes,パナソニック株式会社,綾瀬はる...
review_number_to_zero    綾瀬はるか meets Beautiful Athletes,パナソニック株式会社,綾瀬はる...
lsbw                     綾瀬はるか * * * * パナソニック * 綾瀬はるか * * * * パナソニック * ...
Name: 302864, dtype: object
-----
0.46526332758166034
test                     Document 2015-2018 綾瀬はるかフォトブック,綾瀬　はるか,Document...
review_number_to_zero    Document 0-0 綾瀬はるかフォトブック,綾瀬　はるか,Document 0-0 綾...
lsbw                     * * * * 綾瀬はるか フォトブック * 綾瀬 はるか * * * * * 綾瀬はるか ...
Name: 372852, dtype: object
-----
0.44505487920963005
test                     景気をよくする人気女優綾瀬はるかの成功術,大川隆法,景気をよくする人気女優綾瀬はるかの成功術...
review_number_to_zero    景気をよくする人気女優綾瀬はるかの成功術,大川隆法,景気をよくする人気女優綾瀬はるかの成功術...
lsbw                     景気 人気 女優 綾瀬はるか 成功 術 * 大川隆法 * 景気 人気 女優 綾瀬はるか 成功...
Name: 220714, dtype: object
-----
-----
-----
-----
-----
0.534874662649994
test                     焦眉 警視庁強行犯係・樋口顕,今野 敏,焦眉 警視庁強行犯係・樋口顕,今野 敏,焦眉 警視庁...
review_number_to_zero    焦眉 警視庁強行犯係・樋口顕,今